In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
import gc
from tqdm.auto import tqdm 


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train.parquet
/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train_multi.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet


In [2]:
%%time
train = pl.read_parquet('/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train.parquet')
train.head()

CPU times: user 2.09 s, sys: 1.25 s, total: 3.34 s
Wall time: 2.04 s


series_id,step,timestamp,anglez,enmo,awake
str,u32,str,f32,f32,i64
"""08db4255286f""",0,"""2018-11-05T10:…",-30.845301,0.0447,1
"""08db4255286f""",1,"""2018-11-05T10:…",-34.181801,0.0443,1
"""08db4255286f""",2,"""2018-11-05T10:…",-33.877102,0.0483,1
"""08db4255286f""",3,"""2018-11-05T10:…",-34.282101,0.068,1
"""08db4255286f""",4,"""2018-11-05T10:…",-34.385799,0.0768,1


In [3]:
train.replace("awake", train["awake"].cast(pl.Int8))

/tmp/ipykernel_21/2868793369.py:1: DeprecationWarning: `replace` is deprecated. DataFrame.replace is deprecated and will be removed in a future version. Please use
    df = df.with_columns(new_column.alias(column_name))
instead.
  train.replace("awake", train["awake"].cast(pl.Int8))


series_id,step,timestamp,anglez,enmo,awake
str,u32,str,f32,f32,i8
"""08db4255286f""",0,"""2018-11-05T10:…",-30.845301,0.0447,1
"""08db4255286f""",1,"""2018-11-05T10:…",-34.181801,0.0443,1
"""08db4255286f""",2,"""2018-11-05T10:…",-33.877102,0.0483,1
"""08db4255286f""",3,"""2018-11-05T10:…",-34.282101,0.068,1
"""08db4255286f""",4,"""2018-11-05T10:…",-34.385799,0.0768,1
"""08db4255286f""",5,"""2018-11-05T10:…",-34.925598,0.0511,1
"""08db4255286f""",6,"""2018-11-05T10:…",-30.513399,0.1073,1
"""08db4255286f""",7,"""2018-11-05T10:…",-30.509399,0.0649,1
"""08db4255286f""",8,"""2018-11-05T10:…",-32.8806,0.0485,1


In [4]:
(train["step"][1:]-train["step"][:-1]).ne(1).any()

True

In [5]:
for series_id in train["series_id"].unique():
    temp = train.filter(pl.col("series_id") == series_id)
    print((temp["step"][1:]-temp["step"][:-1]).ne(1).any())

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [6]:
df = train.clone()
grouped_df = df.group_by("series_id").agg(pl.col("step").diff())
grouped_df

series_id,step
str,list[i64]
"""d150801f3145""","[null, 1, … 1]"
"""0a96f4993bd7""","[null, 1, … 1]"
"""c6788e579967""","[null, 1, … 1]"
"""89bd631d1769""","[null, 1, … 1]"
"""3664fe9233f9""","[null, 1, … 1]"
"""8e32047cbc1f""","[null, 1, … 1]"
"""d515236bdeec""","[null, 1, … 1]"
"""1087d7b0ff2e""","[null, 1, … 1]"
"""7822ee8fe3ec""","[null, 1, … 1]"


In [7]:
def luan_unique(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]


ids = luan_unique(train['series_id'])

print(ids)

len(ids)

['08db4255286f', '0a96f4993bd7', '0cfc06c129cc', '1087d7b0ff2e', '10f8bc1f7b07', '18b61dd5aae8', '29c75c018220', '3452b878e596', '349c5562ee2c', '3664fe9233f9', '483d6545417f', '55a47ff9dc8a', '5acc9d63b5fd', '5f94bb3e1bed', '655f19eabf1e', '67f5fc60e494', '72bbd1ac3edf', '76237b9406d5', '7822ee8fe3ec', '89bd631d1769', '8e32047cbc1f', '939932f1822d', '9ee455e4770d', 'a9a2f7fac455', 'a9e5f5314bcb', 'af91d9a50547', 'b364205aba43', 'c535634d7dcd', 'c6788e579967', 'c68260cc9e8f', 'ca730dbf521d', 'd150801f3145', 'd25e479ecbb7', 'd515236bdeec', 'd5e47b94477e']


35

smooth data

In [8]:
# %%time

# dict_ids2data = {}

# for i in tqdm(range(len(ids))):
# # for i in range(2):
#     print (i,': ', ids[i])
#     dict_ids2data[ids[i]] = train.filter(pl.col("series_id") == ids[i])
    
#     dict_ids2data[ids[i]] = dict_ids2data[ids[i]].with_columns(
#         anglez = pl.col("anglez").rolling_mean(window_size=21, center = True),
#         enmo = pl.col("enmo").rolling_mean(window_size=21, center = True),
#     )
    
#     dict_ids2data[ids[i]] = dict_ids2data[ids[i]].fill_null(strategy="forward").fill_null(strategy="backward")
    


# train_smooth = pl.concat([i for i in dict_ids2data.values()], rechunk=True)


# del dict_ids2data
# gc.collect()

In [9]:
train_smooth = train


In [10]:
# train_reduce = train_smooth.take_every(5)

In [11]:
train_smooth["timestamp"].str.slice(-5,).unique()

timestamp
str
"""-0500"""
"""-0400"""


In [12]:
train_smooth["timestamp"]

timestamp
str
"""2018-11-05T10:…"
"""2018-11-05T10:…"
"""2018-11-05T10:…"
"""2018-11-05T10:…"
"""2018-11-05T10:…"
"""2018-11-05T10:…"
"""2018-11-05T10:…"
"""2018-11-05T10:…"
"""2018-11-05T10:…"


In [13]:
%%time

def process_timestamp(df):
    df = df.with_columns(
        #year = (df["timestamp"].str.slice(0,4).cast(pl.Int16) - 2000).cast(pl.UInt8),
        #month = df["timestamp"].str.slice(5,2).cast(pl.Int8),
        #day = df["timestamp"].str.slice(8,2).cast(pl.Int8),
        #hour = df["timestamp"].str.slice(11,2).cast(pl.Int8),
        #minute = df["timestamp"].str.slice(14,2).cast(pl.Int8),
        #second = df["timestamp"].str.slice(17,2).cast(pl.Int8),
        #time_zone = df["timestamp"].str.slice(-5,3).cast(pl.Int32),
        enmo =  (pl.col('enmo')*1000).cast(pl.UInt16),
        date_time =pl.col('timestamp').str.strptime(pl.Datetime),
    )
    return df
    
    
# print (process_timestamp(train_smooth))

train_time_extract = process_timestamp(train_smooth)

train_time_extract

CPU times: user 11.2 s, sys: 282 ms, total: 11.5 s
Wall time: 11.3 s


series_id,step,timestamp,anglez,enmo,awake,date_time
str,u32,str,f32,u16,i8,"datetime[μs, UTC]"
"""08db4255286f""",0,"""2018-11-05T10:…",-30.845301,44,1,2018-11-05 14:00:00 UTC
"""08db4255286f""",1,"""2018-11-05T10:…",-34.181801,44,1,2018-11-05 14:00:05 UTC
"""08db4255286f""",2,"""2018-11-05T10:…",-33.877102,48,1,2018-11-05 14:00:10 UTC
"""08db4255286f""",3,"""2018-11-05T10:…",-34.282101,68,1,2018-11-05 14:00:15 UTC
"""08db4255286f""",4,"""2018-11-05T10:…",-34.385799,76,1,2018-11-05 14:00:20 UTC
"""08db4255286f""",5,"""2018-11-05T10:…",-34.925598,51,1,2018-11-05 14:00:25 UTC
"""08db4255286f""",6,"""2018-11-05T10:…",-30.513399,107,1,2018-11-05 14:00:30 UTC
"""08db4255286f""",7,"""2018-11-05T10:…",-30.509399,64,1,2018-11-05 14:00:35 UTC
"""08db4255286f""",8,"""2018-11-05T10:…",-32.8806,48,1,2018-11-05 14:00:40 UTC


In [14]:
train_time_extract["date_time"][1:]-train_time_extract["date_time"][:-1]

date_time
duration[μs]
5s
5s
5s
5s
5s
5s
5s
5s
5s


In [15]:
(train_time_extract["date_time"][1:]-train_time_extract["date_time"][:-1]).map_elements(lambda x: (x.seconds  / 60) > 1).unique()

date_time
bool
false
true


In [16]:
for series_id in train["series_id"].unique():
    temp = train_time_extract.filter(pl.col("series_id") == series_id)
    # print((temp["date_time"][1:]-temp["date_time"][:-1]).map_elements(lambda x: x.days > 1).any())
    my_series = (temp["date_time"][1:]-temp["date_time"][:-1]).map_elements(lambda x: x.seconds < 5)
    """
    seconds = (temp["date_time"][1:]-temp["date_time"][:-1]).map_elements(lambda x: x.seconds )
    days = (temp["date_time"][1:]-temp["date_time"][:-1]).map_elements(lambda x: x.days )
    print(seconds.unique())
    print(days.unique())
    continue
    """
    b = pl.Series("b", [False])
    assign_series = my_series.extend(b) 
    temp = temp.with_columns(
        time_stamp = assign_series,
    )
    #filtered_df = 
    print(temp.filter(pl.col("time_stamp")))

shape: (0, 8)
┌───────────┬──────┬───────────┬────────┬──────┬───────┬───────────────────┬────────────┐
│ series_id ┆ step ┆ timestamp ┆ anglez ┆ enmo ┆ awake ┆ date_time         ┆ time_stamp │
│ ---       ┆ ---  ┆ ---       ┆ ---    ┆ ---  ┆ ---   ┆ ---               ┆ ---        │
│ str       ┆ u32  ┆ str       ┆ f32    ┆ u16  ┆ i8    ┆ datetime[μs, UTC] ┆ bool       │
╞═══════════╪══════╪═══════════╪════════╪══════╪═══════╪═══════════════════╪════════════╡
└───────────┴──────┴───────────┴────────┴──────┴───────┴───────────────────┴────────────┘
shape: (0, 8)
┌───────────┬──────┬───────────┬────────┬──────┬───────┬───────────────────┬────────────┐
│ series_id ┆ step ┆ timestamp ┆ anglez ┆ enmo ┆ awake ┆ date_time         ┆ time_stamp │
│ ---       ┆ ---  ┆ ---       ┆ ---    ┆ ---  ┆ ---   ┆ ---               ┆ ---        │
│ str       ┆ u32  ┆ str       ┆ f32    ┆ u16  ┆ i8    ┆ datetime[μs, UTC] ┆ bool       │
╞═══════════╪══════╪═══════════╪════════╪══════╪═══════╪════════════════

In [17]:
type(train_time_extract)

polars.dataframe.frame.DataFrame

In [18]:
train_time_extract.select(pl.corr(["anglez","enmo","hour"]))

TypeError: corr() missing 1 required positional argument: 'b'

In [ ]:
%%time

features, feature_cols = [], []


for mins in [5, 30, 60*2, 60*8] :
    for var in ['enmo', 'anglez'] :
        features += [
            pl.col(var).rolling_mean(12 * mins, center=True, min_periods=1).abs().cast(pl.UInt16).alias(f'{var}_{mins}m_mean'),
            pl.col(var).rolling_max(12 * mins, center=True, min_periods=1).abs().cast(pl.UInt16).alias(f'{var}_{mins}m_max'),
            pl.col(var).rolling_std(12 * mins, center=True, min_periods=1).abs().cast(pl.UInt16).alias(f'{var}_{mins}m_std')
        ]

        feature_cols += [ 
            f'{var}_{mins}m_mean', f'{var}_{mins}m_max', f'{var}_{mins}m_std'
        ]

        # Getting first variations
        features += [
            (pl.col(var).diff().abs().rolling_mean(12 * mins, center=True, min_periods=1)*10).abs().cast(pl.UInt32).alias(f'{var}_1v_{mins}m_mean'),
            (pl.col(var).diff().abs().rolling_max(12 * mins, center=True, min_periods=1)*10).abs().cast(pl.UInt32).alias(f'{var}_1v_{mins}m_max'),
            (pl.col(var).diff().abs().rolling_std(12 * mins, center=True, min_periods=1)*10).abs().cast(pl.UInt32).alias(f'{var}_1v_{mins}m_std')
        ]

        feature_cols += [ 
            f'{var}_1v_{mins}m_mean', f'{var}_1v_{mins}m_max', f'{var}_1v_{mins}m_std'
        ]

# id_cols = ['series_id', 'step', 'timestamp']


dict_ids2data = {}

for i in tqdm(range(len(ids))):
# for i in range(2):
    print (i,': ', ids[i])
    dict_ids2data[ids[i]] = train_time_extract.filter(pl.col("series_id") == ids[i])
    
    dict_ids2data[ids[i]] = dict_ids2data[ids[i]].with_columns(
        features
    )
    
    dict_ids2data[ids[i]] = dict_ids2data[ids[i]].fill_null(strategy="forward").fill_null(strategy="backward")
    


train_data = pl.concat([i for i in dict_ids2data.values()], rechunk=True)


del dict_ids2data
gc.collect()

train_data

In [ ]:
c_types = train_data.dtypes
c_name = train_data.columns

print(len(c_name))
print('\n'.join(["{}\t-\t{}".format(str(a_), str(b_)) for a_, b_ in zip(c_name, c_types)]))

In [ ]:
import joblib
joblib.dump(train_data, 'train_luan_fe.pkl')


In [ ]:
%%time
train_data.write_csv('train_luan.csv')
train_data.write_parquet('train_luan.parquet')

In [ ]:
# data_dtype = {
#         "series_id": "category",
#         "step": "uint32",
#         "timestamp": "str",
#         "anglez": "float32",
#         "enmo": "float32",
#     }
# data = pd.read_parquet('/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet').astype(data_dtype, copy=False)